# nb-model_xg-model-data-fi

In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
from collections import OrderedDict
from functools import partial, reduce
import logging

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname

def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fname = 'nb-model_xg-model-data-fi.ipynb'
dir_name = 'model'
fix_path(get_cwd(fname, dir_name +sep))

import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from dask import delayed, compute
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.dataset import Dataset as TorchDataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm
import torchfunc
from torchmeta.utils.data import BatchMetaDataLoader
import pytorch_lightning as pl

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', 50)

from common_util import MODEL_DIR, RECON_DIR, JSON_SFX_LEN, DT_CAL_DAILY_FREQ, pd_to_np, pairwise, df_midx_restack, compose, is_type, df_rows_in_year, list_all_eq, remove_dups_list, NestedDefaultDict, set_loglevel, search_df, chained_filter, get_variants, load_df, dump_df, load_json, gb_transpose, pd_common_index_rows, filter_cols_below, inner_join, outer_join, ser_shift, list_get_dict, window_iter, benchmark
from common_util import pd_split_ternary_to_binary, np_value_counts, isnt, window_iter, all_eq, np_assert_identical_len_dim, pd_idx_rename, midx_get_level, pd_rows, midx_intersect, pd_get_midx_level, pd_common_idx_rows, midx_split, pd_midx_to_arr, window_iter, np_at_least_nd, np_is_ndim, identity_fn
from model.common import DATASET_DIR, XG_PROCESS_DIR, XG_DATA_DIR, XG_DIR, PYTORCH_MODELS_DIR, ERROR_CODE, TEST_RATIO, VAL_RATIO, EXPECTED_NUM_HOURS, default_dataset
from model.common import PYTORCH_ACT_MAPPING, PYTORCH_OPT_MAPPING, PYTORCH_SCH_MAPPING, PYTORCH_LOSS_MAPPING
from model.xg_util import xgload, get_xg_feature_dfs, get_xg_label_target_dfs

from model.preproc_util import temporal_preproc_3d, stride_preproc_3d
from model.train_util import pd_to_np_tvt, batchify
#from model.pl_util import TCNModel
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

CRITICAL:root:script location: /home/kev/crunch/model/nb-model_xg-model-data-fi.ipynb
CRITICAL:root:using project dir: /home/kev/crunch/


Prune the xg data down to the data of interest to use in further experiments.

## Load Data

In [22]:
assets = ['sp_500', 'russell_2000', 'nasdaq_100', 'dow_jones']
chosen_asset = assets[0]

In [23]:
fd = get_xg_feature_dfs(chosen_asset)
ld, td = get_xg_label_target_dfs(chosen_asset)

## Select Data

### Choose Features and Join

In [24]:
for k0, v0 in fd.items():
    print(k0)
    for k1,v1 in fd[k0].items():
        print('\t' +k1)
        for k2, v2 in fd[k0][k1].items():
            print('\t\t' +k2)
            for k3, v3 in fd[k0][k1][k2].items():
                print('\t\t\t' +k3)

d
	pba
		ddiff
			pba_dohlca_ddiff(1)
		dffd
			pba_dohlca_dffd(0.200000,0.010000)_dffd(0.400000,0.010000)_dffd(0.600000,0.010000)_dffd(0.800000,0.010000)
		dlogret
			pba_dlh_dlogret
			pba_doc_dlogret
		dohlca
			pba_dohlca
		dwrmx
			pba_dohlca_dwrmx(-1)_dwrmx(20)_dwrmx(240)_dwrmx(5)
		dwrod
			pba_dohlca_dwrod(-1)_dwrod(20)_dwrod(240)_dwrod(5)
		dwrpt
			pba_dohlca_dwrpt(-1)_dwrpt(20)_dwrpt(240)_dwrpt(5)
		dwrzn
			pba_dohlca_dwrzn(-1)_dwrzn(20)_dwrzn(240)_dwrzn(5)
	vol
		ddiff
			vol_dohlca_ddiff(1)
		dffd
			vol_dohlca_dffd(0.200000,0.010000)_dffd(0.400000,0.010000)_dffd(0.600000,0.010000)_dffd(0.800000,0.010000)
		dlogret
			vol_dlh_dlogret
			vol_doc_dlogret
		dohlca
			vol_dohlca
		dwrmx
			vol_dohlca_dwrmx(-1)_dwrmx(20)_dwrmx(240)_dwrmx(5)
		dwrod
			vol_dohlca_dwrod(-1)_dwrod(20)_dwrod(240)_dwrod(5)
		dwrpt
			vol_dohlca_dwrpt(-1)_dwrpt(20)_dwrpt(240)_dwrpt(5)
		dwrzn
			vol_dohlca_dwrzn(-1)_dwrzn(20)_dwrzn(240)_dwrzn(5)
	buzz
		ddiff
			trmi3_etf3_news_buzz_dohlca_ddiff(1)_

In [25]:
chosen_f = fd['h']['pba']['hdzn']['pba_hohlca_hdzn']

In [26]:
chosen_f

9         10        11  \
id0        id1                                                          
1996-01-02 pba_open_pba_hohlca_hdzn     -0.887489 -1.927157  0.604677   
           pba_high_pba_hohlca_hdzn     -1.942512  0.129960 -0.028402   
           pba_low_pba_hohlca_hdzn      -1.660850 -1.296166  0.105451   
           pba_close_pba_hohlca_hdzn    -2.076113  0.266295 -0.379717   
           pba_avgPrice_pba_hohlca_hdzn -2.056411 -0.190552 -0.102288   
...                                           ...       ...       ...   
2018-01-08 pba_open_pba_hohlca_hdzn     -0.517945 -1.152933 -0.980733   
           pba_high_pba_hohlca_hdzn     -1.126055 -1.281708 -0.670769   
           pba_low_pba_hohlca_hdzn      -1.452088 -1.094826 -0.570273   
           pba_close_pba_hohlca_hdzn    -1.268278 -1.134863 -0.918063   
           pba_avgPrice_pba_hohlca_hdzn -1.254814 -1.213812 -0.693628   

                                               12        13        14  \
id0        id1                                                          
1996-01-02 pba_open_pba_hohlca_hdzn     -0.084843  0.103697  0.421523   
           pba_high_pba_hohlca_hdzn     -0.627422 -0.062828  0.219469   
           pba_low_pba_hohlca_hdzn       0.061513  0.351503  0.615130   
           pba_close_pba_hohlca_hdzn    -0.182461  0.088765  0.113422   
           pba_avgPrice_pba_hohlca_hdzn -0.248642  0.153727  0.379935   
...                                           ...       ...       ...   
2018-01-08 pba_open_pba_hohlca_hdzn     -0.736783 -0.019283  1.042617   
           pba_high_pba_hohlca_hdzn     -0.530681  0.710655  0.916895   
           pba_low_pba_hohlca_hdzn      -0.269719  0.379592  1.000548   
           pba_close_pba_hohlca_hdzn    -0.240981  0.749626  0.946414   
           pba_avgPrice_pba_hohlca_hdzn -0.406994  0.646859  0.941775   

                                               15        16  
id0        id1                                               
1996-01-02 pba_open_pba_hohlca_hdzn      0.453845  1.315746  
           pba_high_pba_hohlca_hdzn      0.942424  1.369312  
           pba_low_pba_hohlca_hdzn       0.474529  1.348892  
           pba_close_pba_hohlca_hdzn     0.902443  1.267366  
           pba_avgPrice_pba_hohlca_hdzn  0.605459  1.458772  
...                                           ...       ...  
2018-01-08 pba_open_pba_hohlca_hdzn      1.265042  1.100017  
           pba_high_pba_hohlca_hdzn      1.146484  0.835177  
           pba_low_pba_hohlca_hdzn       0.742525  1.264242  
           pba_close_pba_hohlca_hdzn     0.889712  0.976432  
           pba_avgPrice_pba_hohlca_hdzn  0.851456  1.129157  

[27405 rows x 8 columns]

### Choose Labels/Targets and Process

In [27]:
chosen_l = pd_split_ternary_to_binary(ld['hoc']['pba']['ddir'].replace(to_replace=-1, value=0))
chosen_t = pd_split_ternary_to_binary(td['hoc']['pba']['dret'])

### Get Common Indexed Rows (Intersect First Level of MultiIndex)

In [28]:
year_interval = ('2009', '2018')
common_idx = midx_intersect(pd_get_midx_level(chosen_f), pd_get_midx_level(chosen_l), pd_get_midx_level(chosen_t))
common_idx = common_idx[(common_idx > year_interval[0]) & (common_idx < year_interval[1])]
feature_df, label_df, target_df = map(compose(partial(pd_rows, idx=common_idx), df_midx_restack), [chosen_f, chosen_l, chosen_t])
assert(all(feature_df.index.levels[0]==label_df.index.levels[0]))
assert(all(feature_df.index.levels[0]==target_df.index.levels[0]))

## PL TCN Model

In [29]:
from model.pl_tcn import TCNModel

In [30]:
m_params = {
	'window_size': 10,
	'num_blocks': 1,
	#'block_channels': [[30, 20, 10]],
	'block_channels': [[5]],
	'block_act': 'elu',
	'out_act': 'relu',
	'kernel_sizes': [3],
	'dilation_index': 'global',
	'global_dropout': .2,
	'no_dropout': [0],
	'out_shape': len(chosen_l.columns)*len(chosen_l.index.levels[1]) # directions * slots per direction
}
t_params = {
	'epochs': 200,
	'batch_size': 64*2*2,
	'loss': 'ce',
	'opt': {
		'name': 'adam',
		'kwargs': {
			'lr': .001
		}
	},
	'sch': {
		'name': 'rpl',
		'kwargs': {
			'mode': 'min',
			'factor': 0.1,
			'patience': 10,
			'threshold': 0.0001,
			'threshold_mode': 'rel',
			'cooldown': 0,
			'min_lr': 0
		}
	}
}

In [31]:
mdl = TCNModel(m_params, t_params, (feature_df, label_df, target_df))

In [32]:
mdl

TCNModel(
  (loss): CrossEntropyLoss()
  (clf): OutputLinear(
    (emb): TemporalConvNet(
      (convnet): Sequential(
        (RB_0): ResidualBlock(
          (net): Sequential(
            (TL_0_0): TemporalLayer1d(
              (layer): Sequential(
                (0): Conv1d(5, 40, kernel_size=(3,), stride=(1,), padding=(1,))
                (1): ELU(alpha=1.0)
                (2): Dropout(p=0, inplace=False)
              )
            )
          )
          (downsample): Conv1d(5, 40, kernel_size=(1,), stride=(1,))
          (out_act): ReLU()
        )
      )
    )
    (out): Linear(in_features=40, out_features=2, bias=True)
  )
)

In [33]:
trainer = pl.Trainer(max_nb_epochs=t_params['epochs'], gpus=1, amp_level='O1', use_amp=True)

In [34]:
trainer.fit(mdl)

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


1

## Debug Data (Runtime Transforms)

### Split into Train/Val/Test and Convert to Numpy Tensor

In [608]:
train_np, val_np, test_np = zip(*map(pd_to_np_tvt, (feature_df, label_df, target_df)))
shapes = np.asarray(tuple(map(lambda tvt: tuple(map(np.shape, tvt)), (train_np, val_np, test_np))))
assert all(np.array_equal(a[:, 1:], b[:, 1:]) for a, b in pairwise(shapes)), 'feature, label, target shapes must be identical across splits'
assert all(len(np.unique(mat.T[0, :]))==1 for mat in shapes), 'first dimension (N) must be identical length in each split for all (feature, label, and target) tensors'

### Runtime Preproc

In [609]:
params = {
    'loss': 'ce',
    'batch_size': 1,
    'window_size': 20,
}

In [610]:
def __preproc__(data, m_params, overlap=True):
    x, y, z = temporal_preproc_3d(data, window_size=m_params['window_size'], apply_idx=[0]) if (overlap) else stride_preproc_3d(data, window_size=m_params['window_size'])
    if (m_params['loss'] in ('bce', 'bcel', 'ce', 'nll')):
        y_new = np.sum(y, axis=(1, 2), keepdims=False)		# Sum label matrices to scalar values
        if (y.shape[1] > 1):
            y_new += y.shape[1]								# Shift to range [0, C-1]
        if (m_params['loss'] in ('bce', 'bcel')):
            y_new = np.expand_dims(y_new, axis=-1)
        y = y_new
    return (x, y, z)

#@pl.data_loader
def train_dataloader(t_params, flt):
    logging.info('train_dataloader called')
    return batchify(t_params, __preproc__(flt), False)

#@pl.data_loader
def val_dataloader(t_params, flt):
    logging.info('val_dataloader called')
    return batchify(t_params, __preproc__(flt), False)

#@pl.data_loader
def test_dataloader(t_params, flt):
    logging.info('test_dataloader called')
    return batchify(t_params, __preproc__(flt), False)

### Overlapping Episodes:

In [611]:
train_ol_np = __preproc__(train_np, params)
val_ol_np = __preproc__(val_np, params)
test_ol_np = __preproc__(test_np, params)
print(tuple(map(lambda tvt: tuple(map(np.shape, tvt)), (train_np, val_np, test_np)))[0])
print(tuple(map(np.shape, train_ol_np)))
print(np_value_counts(train_ol_np[1]))

((1359, 3, 8), (1359, 1, 2), (1359, 1, 2))
((1340, 3, 160), (1340,), (1340, 1, 2))
(array([0., 1.]), array([593, 747]))


### Non-Overlapping Episodes:

In [612]:
train_nol_np = __preproc__(train_np, params, False)
val_nol_np = __preproc__(val_np, params, False)
test_nol_np = __preproc__(test_np, params, False)
print(tuple(map(lambda tvt: tuple(map(np.shape, tvt)), (train_np, val_np, test_np)))[0])
print(tuple(map(np.shape, train_nol_np)))

((1359, 3, 8), (1359, 1, 2), (1359, 1, 2))
((67, 3, 20, 8), (67, 2), (67, 1, 20, 2))


In [613]:
def batchify(params, data, shuffle_batches=False):
	"""
	Return a torch.DataLoader made from a tuple of numpy arrays.

	Args:
		params (dict): model parameters dictionary
		data (tuple): tuple of numpy arrays, features are the first element
		shuffle_batches (bool): whether or not to shuffle the batches

	Returns:
		torch.DataLoader
	"""
	f = torch.tensor(data[0], dtype=torch.float32, requires_grad=True)
	if (params['loss'] in ('bce', 'bcel', 'mae', 'mse')):
		l = [torch.tensor(d, dtype=torch.float32, requires_grad=False) for d in data[1:]]
	elif (params['loss'] in ('ce', 'nll')):
		l = [torch.tensor(d, dtype=torch.int64, requires_grad=False).squeeze() for d in data[1:]]
	ds = TensorDataset(f, *l)
	dl = DataLoader(ds, batch_size=params['batch_size'], shuffle=shuffle_batches)
	return ds

## Sklearn Tests

In [614]:
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, ElasticNet
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, StackingClassifier

In [615]:
x_train, y_train, z_train = train_ol_np
x_val, y_val, z_val = val_ol_np
x_test, y_test, z_test = test_ol_np

In [616]:
def np_collapse_last_two_dim(arr):
    return arr.reshape(arr.shape[0], arr.shape[1]*arr.shape[2])

In [617]:
x_train.reshape(1340, 800)

ValueError: cannot reshape array of size 643200 into shape (1340,800)

In [618]:
x_train_new = np_collapse_last_two_dim(x_train)
x_val_new = np_collapse_last_two_dim(x_val)
x_test_new = np_collapse_last_two_dim(x_test)

In [619]:
# Logistic Regression
clf = LogisticRegression(C=10**-2, l1_ratio=.9, penalty='elasticnet', random_state=0, solver='saga').fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.5092165898617511
diff : 0.000000%


In [620]:
# SGDClassifier
clf = SGDClassifier(alpha=10**-2, l1_ratio=.8, loss='log', penalty='elasticnet', random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.5069124423963134
diff : -0.230415%


In [621]:
# SGDClassifier
clf = SGDClassifier(alpha=10**-5, l1_ratio=.1, loss='modified_huber', max_iter=1000, penalty='elasticnet', random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.5230414746543779
diff : 1.382488%


In [622]:
# RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, criterion='gini', bootstrap=True, max_depth=4, min_impurity_decrease=0, random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.5069124423963134
diff : -0.230415%


In [635]:
# RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, criterion='entropy', bootstrap=True, max_depth=None,
                             min_impurity_decrease=0, random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.4769585253456221
diff : -3.225806%


In [624]:
# RandomForestClassifier
clf = RandomForestClassifier(n_estimators=7, class_weight='balanced_subsample', criterion='entropy',
                             bootstrap=True, max_depth=3, min_impurity_decrease=0, random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.4930875576036866
diff : -1.612903%


In [625]:
# RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1000, class_weight='balanced_subsample', criterion='entropy',
                             bootstrap=True, max_depth=4, min_impurity_decrease=0, random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.49078341013824883
diff : -1.843318%


In [626]:
# GradientBoostingClassifier
clf = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100, subsample=1.0,
                                 criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1,
                                 min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.4861751152073733
diff : -2.304147%


In [627]:
# GradientBoostingClassifier
clf = GradientBoostingClassifier(loss='exponential', learning_rate=0.1, n_estimators=100, subsample=1.0,
                                 criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1,
                                 min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.4861751152073733
diff : -2.304147%


In [628]:
# HistGradientBoostingClassifier
clf = HistGradientBoostingClassifier(loss='auto', learning_rate=0.1, max_iter=100, max_leaf_nodes=31,
                                     max_depth=4, min_samples_leaf=20, l2_regularization=0.0, max_bins=255, random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.46543778801843316
diff : -4.377880%


In [629]:
# RF Alt 1
clf1 = DecisionTreeClassifier(criterion='entropy', max_depth=3, max_features='auto', class_weight='balanced')
clf = BaggingClassifier(base_estimator=clf1, bootstrap=True, bootstrap_features=False, n_estimators=100, random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.532258064516129
diff : 2.304147%


In [630]:
# RF Alt 2
clf = RandomForestClassifier(n_estimators=1, class_weight='balanced_subsample', criterion='entropy',
                             bootstrap=False, max_depth=3, min_impurity_decrease=0, random_state=0)
clf = BaggingClassifier(base_estimator=clf1, bootstrap=True, bootstrap_features=False, n_estimators=1000, random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.5069124423963134
diff : -0.230415%


In [598]:
random_states = range(10)
results = np.zeros(len(random_states))
y_mean = y_val.mean()
for r in random_states:
        # RandomForestClassifier
    clf = RandomForestClassifier(n_estimators=100, class_weight=None, criterion='entropy',
                                 bootstrap=True, max_depth=None, min_impurity_decrease=0, random_state=r).fit(x_train_new, y_train)
    sc = clf.score(x_val_new, y_val)
    diff = sc-y_mean
    results[r] = diff*100
    #print('base :', y_val.mean())
    #print('score:', sc)
    #print('diff : {:%}'.format((sc-y_val.mean())))
print(results)
print(results.mean())

[ 2.76497696 -2.99539171 -3.4562212   2.07373272  1.38248848 -1.61290323
 -0.92165899  0.46082949 -1.84331797 -3.68663594]
-0.783410138248844


In [67]:
# Few Shot Tests
# XXX - Need to update for code changes
tr_split = .75
w = params['window_size']
tr, ts = int(tr_split*w), (1-tr_split)*w
print(tr, ts)

scs = []
for x, y, z in zip(*train_nol_np):
    x, y = x.T, y
    print(x.shape)
    print(x[0:tr].shape)
    print(x[tr:].shape)
    break
    clf = LogisticRegression(C=10**-2, l1_ratio=.9, penalty='elasticnet', random_state=0, solver='saga').fit(x[0:tr], y[0:tr].squeeze())
    #clf = LinearRegression().fit(x[0:tr], y[0:tr].squeeze())
    #yh = clf.predict(x[tr:])
    #|yp = clf.predict_proba(x[tr:])
    sc = clf.score(x[tr:], y[tr:].squeeze())
    scs.append(sc)
    #print(sc)
    #print(x)
    #print(y.squeeze())
    #print(z.T)

15 5.0
(160, 5)
(15, 5)
(145, 5)


In [68]:
(np.array(scs).mean()-train_nol[1].squeeze().mean())*100

/home/kev/crunch/model/nb-model_xg-model-data-fi.ipynb:1: RuntimeWarning: Mean of empty slice.
  {
/home/kev/miniconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


NameError: name 'train_nol' is not defined